In [1]:
%pip install pandas
%pip install sqlalchemy
%pip install scikit-learn
%pip install imblearn
%pip install matplotlib
%pip install seaborn
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.




In [2]:
import warnings
import time
import sys
import datetime

import numpy as np 
import pandas as pd 

warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def read_data(input_file):
    """
    指定されたファイルからデータを読み込み、前処理を行う。

    Parameters
    ----------
    input_file : str
        読み込むデータファイルのパス。

    Returns
    -------
    pd.DataFrame
        前処理されたデータフレーム。
    """
    
    df = pd.read_csv(input_file)
    # df['first_active_month'] = pd.to_datetime(df['first_active_month'], format='%b-%d')
    # df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days
    return df

In [4]:
train = read_data('../data/raw/train.csv')
test = read_data('../data/raw/test.csv')
merchants = read_data('../data/raw/merchants.csv')


new_transactions = pd.read_csv('../data/raw/new_merchant_transactions.csv',
                               parse_dates=['purchase_date'])

historical_transactions = pd.read_csv('../data/raw/historical_transactions.csv',
                                      parse_dates=['purchase_date'])



# historical_transactions = binarize(historical_transactions)
# new_transactions = binarize(new_transactions)

In [5]:
print(historical_transactions.columns)

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id'],
      dtype='object')


In [6]:
new_train = read_data('../data/processed/processed20240614_train.csv')
new_test = read_data('../data/processed/processed20240614_test.csv')

In [7]:
table = new_train

missing_counts = table.isna().sum()

print(f'テーブルの行数：{len(table)}')
print('テーブルの欠損値\n' + str(missing_counts))

テーブルの行数：201917
テーブルの欠損値
first_active_month                   0
card_id                              0
feature_1                            0
feature_2                            0
feature_3                            0
                                 ...  
city_id_purchase_amount_std     155121
category_1_installments_mean    142076
category_1_installments_min     142076
category_1_installments_max     142076
category_1_installments_std     190727
Length: 164, dtype: int64


In [10]:
name_counts = historical_transactions['month_lag'].value_counts()
name_counts

month_lag
-2     3857276
-1     3627430
 0     3470946
-3     3095651
-4     2639805
-5     2364790
-6     2050048
-7     1769844
-8     1427846
-9     1234724
-10    1085053
-11    1007878
-12     791973
-13     689097
Name: count, dtype: int64

In [ ]:
table = new_test

missing_counts = table.isna().sum()

print(f'テーブルの行数：{len(table)}')
print('テーブルの欠損値\n' + str(missing_counts))

テーブルの行数：123623
テーブルの欠損値
first_active_month                   1
card_id                              0
feature_1                            0
feature_2                            0
feature_3                            0
                                 ...  
city_id_purchase_amount_std      94986
category_1_installments_mean     86966
category_1_installments_min      86966
category_1_installments_max      86966
category_1_installments_std     116924
Length: 163, dtype: int64


In [8]:
historical_transactions.head(100)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Y,C_ID_4e6213e9bc,333,N,0,A,278,M_ID_7329fe90b9,0,-0.708019,2018-02-04 13:43:21,1.0,9,37
96,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-6,-0.738538,2017-08-11 10:40:29,1.0,16,34
97,Y,C_ID_4e6213e9bc,88,N,0,A,278,M_ID_21e1552dab,-6,-0.734887,2017-08-11 13:02:59,1.0,16,37
98,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-6,-0.737892,2017-08-04 15:48:28,1.0,16,37
